<a href="https://colab.research.google.com/github/geopayme/Android-Merchant-App/blob/master/PDF_Signer_And_Hasher_Auth_Audit_WORKING_VERSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛡️ PDF_Signer_And_Hasher (Auth + Audit + Email Extraction Fixed)
**Kapodistrian Internal Signing Tools** – Secure + Smart Version

Includes:
- 🔐 Local signer authentication
- 🧠 Auto extraction of email with override
- ✍️ Visible footer with full SHA256
- 🧾 Audit log of each signing

In [19]:
# 🔧 Install PyMuPDF
!pip install --quiet PyMuPDF

In [20]:
# 📤 Upload PDF
from google.colab import files
uploaded = files.upload()
pdf_filename = [f for f in uploaded if f.endswith('.pdf')][0]

Saving preprints-154933-presentation.pdf to preprints-154933-presentation.pdf


In [21]:
from datetime import datetime  # ✅ Fix: ensure datetime is defined

import fitz
import re

doc = fitz.open(pdf_filename)
full_text = "\n".join([page.get_text() for page in doc])

email_match = re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", full_text)
auto_email = email_match.group(0).strip() if email_match else ""

author = input("Author name: ").strip()
title = input("Short title: ").strip()
version = input("Version (e.g. v1): ").strip()
orcid = input("ORCID: ").strip()
institution = input("Institution: ").strip()
email = input(f"Email [{auto_email}]: ").strip() or auto_email

date_str = datetime.today().strftime('%Y-%m-%d')
kas_filename = f"{author}_{title}_{version}_SIGNED_KAS_{date_str}.pdf"


Author name: Antonios Valamontes 
Short title: 
Version (e.g. v1): v3
ORCID: 0009-0008-5616-7746
Institution: Kapodistrian Academy of Science
Email [avalamontes@Kapodistrian.edu.gr]: 


In [22]:
# ✍️ Add footer and hash
from datetime import datetime
import hashlib

# Full UTC timestamp
timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S UTC')

# Save the PDF temporarily to compute hash
temp_path = "_temp_original.pdf"
doc.save(temp_path)

# Compute SHA-256 hash of the saved file
with open(temp_path, 'rb') as f:
    sha256_hash = hashlib.sha256(f.read()).hexdigest()

# Build footer text with placeholder hash replaced
footer_lines = [
    f"Digitally Signed by: {author}",
    f"ORCID: {orcid}",
    f"Institution: {institution}",
    f"Email: {email}",
    f"Timestamp: {timestamp}",
    f"SHA256: {sha256_hash}",
    "Verified by the Kapodistrian Academy of Science"
]
footer_text = "\n".join(footer_lines)

# Define safe rectangle (wider + taller, monospaced layout)
page = doc[-1]
rect = fitz.Rect(50, page.rect.height - 180, page.rect.width - 50, page.rect.height - 30)

# Insert footer into the last page
page.insert_textbox(
    rect,
    footer_text,
    fontsize=7,
    fontname="courier",  # monospaced to prevent overlap
    color=(0, 0, 0)
)

# Save final signed PDF
doc.save(kas_filename)
doc.close()

In [23]:
# 💾 Save .sha256 and .meta.txt
sha_filename = kas_filename + ".sha256"
with open(sha_filename, 'w') as f:
    f.write(sha256_hash)

meta_filename = kas_filename.replace(".pdf", ".meta.txt")
with open(meta_filename, "w") as f:
    f.write(f"Author: {author}\n")
    f.write(f"Title: {title}\n")
    f.write(f"Version: {version}\n")
    f.write(f"Date: {date_str}\n")
    f.write(f"ORCID: {orcid}\n")
    f.write(f"Institution: {institution}\n")
    f.write(f"Email: {email}\n")
    f.write(f"SHA256: {sha256_hash}\n")

In [24]:
from datetime import datetime  # Ensure it's imported

# Manually enter the signer's initials
signer_id = input("Signer ID (e.g., initials or username): ").strip()

# 🧾 Write to audit log
audit_entry = f"[{datetime.utcnow().isoformat()} UTC] Signer: {signer_id}\n"
audit_entry += f"  File: {kas_filename}\n  SHA256: {sha256_hash}\n"
audit_entry += f"  Author: {author}, ORCID: {orcid}, Email: {email}, Institution: {institution}\n"
audit_entry += "  Status: ✅ Signed successfully\n\n"

with open("KAS_audit.log", "a") as f:
    f.write(audit_entry)

print("🔐 Audit entry recorded.")


Signer ID (e.g., initials or username): apv
🔐 Audit entry recorded.


In [25]:
# 📁 Download results
files.download(kas_filename)
files.download(sha_filename)
files.download(meta_filename)
files.download("KAS_audit.log")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>